# <center> Разбор кейса ML-инженера

## Обучим и протестируем модель

In [ ]:
#!pip install lightfm

In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse as sparse
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
import pickle

In [ ]:
ratings = pd.read_csv('data/ratings.csv')
books = pd.read_csv('data/books.csv')
tags = pd.read_csv('data/tags.csv')
book_tags = pd.read_csv('data/book_tags.csv')

In [ ]:
books.head()

In [ ]:
mapper = dict(zip(books.goodreads_book_id,books.book_id))

In [ ]:
tags = pd.read_csv('data/tags_cleaned.csv')
book_tags = book_tags[book_tags.tag_id.isin(tags.tag_id)]
book_tags['id'] = book_tags.goodreads_book_id.apply(lambda x: mapper[x])

In [ ]:
book_tags.head()

In [ ]:
ratings_coo = sparse.coo_matrix((ratings.rating,(ratings.user_id, ratings.book_id)))
feature_ratings  = sparse.coo_matrix(([1]*len(book_tags), (book_tags.id, book_tags.tag_id)))

Объявим вспомогательные константы для обучения модели:

In [ ]:
#число потоков нашего процессора. Ставим 1, так как lightfm на macos ставится без OpenMP
NUM_THREADS = 1

#число параметров вектора 
NUM_COMPONENTS = 60

#число эпох обучения
NUM_EPOCHS = 10 

#зерно датчика случайных чисел
RANDOM_STATE = 42

На этапе создания модели мы используем библиотеку LightFM, чтобы сделать матричное разложение (ALS) наших рейтингов книг и получить два набора векторов. 

In [ ]:
#Разбиваем наш датасет на обучающую и тестовую выборки
train, test = random_train_test_split(ratings_coo, test_percentage=0.2, random_state=RANDOM_STATE)

#Создаём модель
model = LightFM(
    learning_rate=0.05, #темп (скорость) обучения
    loss='warp', #loss-функция
    no_components=NUM_COMPONENTS,#размерность вектора признаков
    random_state=RANDON_STATE #генератор случайных чисел
)

#Обучаем модель
model = model.fit(
    train, #обучающая выборка
    epochs=NUM_EPOCHS, #количество эпох обучения
    num_threads=NUM_THREADS, #количество потоков процессора
    item_features=feature_ratings #признаки товаров (рейтинги книг)
)

Протестируем модель

In [ ]:
#Тестируем нашу модель
precision_score = precision_at_k(
    model, #модель
    test, #тестовая выборка
    num_threads=NUM_THREADS, #количество потоков процессора
    k=10, #количество предложений
    item_features=feature_ratings #признаки товаров
).mean() #усредняем результаты
 
recall_score = recall_at_k(
    model, #модель
    test, #тестовая выборка
    num_threads=NUM_THREADS, #количество потоков процессора
    k=10, #количество предложений
    item_features=feature_ratings #признаки товаров
).mean() #усредняем результаты

print(recall_score, precision_score)

Сохраним модель

In [ ]:
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file, protocol=pickle.HIGHEST_PROTOCOL)

## Добавим эмбеддинги к модели и посмотрим, что получилось

In [ ]:
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)

In [ ]:
# Достаём эбмеддинги
item_biases, item_embeddings = model.get_item_representations(features=feature_ratings)

print(item_biases.shape, item_embeddings.shape)

In [ ]:
#!pip install nmslib

In [ ]:
import nmslib

In [ ]:
#Инициализируем наш граф для поиска
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
 
#Начинаем добавлять наши книги в граф
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

In [ ]:
#Вспомогательная функция для поиска по графу
def nearest_books_nms(book_id, index, n=10):
    nn = index.knnQuery(item_embeddings[book_id], k=n)
    return nn

Найдем id книги 1984

In [ ]:
#Отфильтруем только те, где в названии встречается подстрока "1984"
books[books['title'].apply(lambda x: x.lower().find('1984')) >= 0]

Теперь найдем все похожие книги и посмотрим на них

In [ ]:
#Вызываем функцию для поиска ближайших соседей
print(nearest_books_nms(846, nms_idx))

In [ ]:
#Выделяем идентификаторы рекомендованных книг
nbm = nearest_books_nms(846, nms_idx)[0]
nbm

In [ ]:
#Посмотрим на авторов и названия рекомендованных книг
books[books.book_id.isin(nbm)][['authors', 'title']]

Сохраним эмбеддинги

In [ ]:
with open('item_embeddings.pkl', 'wb') as file:
    pickle.dump(item_embeddings, file, protocol=pickle.HIGHEST_PROTOCOL)